# Devices Report (BMC Discovery)

This notebook reproduces the DisMAL `devices` report using the Tideway Python library to run Discovery Data API searches.
It reads connection details from `config.yaml`, supports an optional `devices_with_cred` filter,
and writes a CSV under the standard `output_<target>` folder.

## Requirements

We use `tideway` (local package in this repo or PyPI), `pandas`, and `PyYAML`.
Uncomment the following to install in your environment if needed.

In [ ]:
# %pip install -q tideway pandas pyyaml

import os, sys, json
from pathlib import Path
import pandas as pd
import yaml
from typing import Any, Dict, List


## Select Appliance (optional)

If your `config.yaml` defines multiple appliances under the `appliances:` list,
set `APPLIANCE_NAME` to one of their names (e.g., 'prod' or 'dev') or use the index.
Defaults to the first appliance if neither is set.

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # integer index if not using name selection

# Optional filter: if set to a credential UUID, runs devices_with_cred flow
DEVICES_WITH_CRED_UUID = None  # e.g., '7636fe3b4bd69466ab487f0000010700'


## Configuration (from config.yaml)

Reads settings from `../config.yaml` including target, token/token_file,
API version, and SSL verification preference.
Saves the CSV to `../output_<target>/devices.csv` (or `devices_with_cred.csv`).

In [ ]:
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
config_path = repo_root / 'config.yaml'
with open(config_path, 'r') as fh:
    cfg = yaml.safe_load(fh) or {}

# Appliance selection
apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target"')

# Token handling: inline token or token file
token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    with open(tf_path, 'r') as tf:
        token = tf.read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

# Version and SSL
API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

# Output path
sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

print('Base Host      :', target)
print('API Version    :', API_VERSION)
print('Verify SSL     :', VERIFY_SSL)
print('Output folder  :', output_dir)
print('Token set      :', bool(token))

# Prefer local Tideway package in this repo if available
local_tideway = repo_root / 'Tideway'
if local_tideway.exists():
    sys.path.insert(0, str(local_tideway))

import importlib
tideway = importlib.import_module('tideway')

API_VERSION_NUM = API_VERSION.lstrip('v')
app = tideway.appliance(target, token, api_version=API_VERSION_NUM, ssl_verify=VERIFY_SSL)
twsearch = app.data()
twcreds = app.credentials()

# Quick probe (optional)
try:
    about = app.api_about
    print('Appliance reachable:', about.status_code)
except Exception as e:
    print('Warning: failed to contact appliance /api/about:', e)


## Helpers (Tideway search, normalization, credentials)

In [ ]:
def list_table_to_json(table_like: List[List[Any]]) -> List[Dict[str, Any]]:
    if not table_like or not isinstance(table_like, list):
        return []
    if not table_like or not isinstance(table_like[0], list):
        return []
    headers = table_like[0]
    rows = table_like[1:]
    out = []
    for r in rows:
        try:
            out.append(dict(zip(headers, r)))
        except Exception:
            # length mismatch or malformed row; skip
            continue
    return out

def to_rows(payload: Any) -> List[Dict[str, Any]]:
    # Tideway bulk object format returns a list: [ [headers...], [row...], ... ]
    if isinstance(payload, list):
        if payload and isinstance(payload[0], list):
            return list_table_to_json(payload)
        # Already list of dicts
        if payload and isinstance(payload[0], dict):
            return payload
        return []
    if hasattr(payload, 'json'):
        try:
            js = payload.json()
        except Exception:
            return []
        # Normalize same as above
        if isinstance(js, list):
            if js and isinstance(js[0], list):
                return list_table_to_json(js)
            if js and isinstance(js[0], dict):
                return js
        if isinstance(js, dict) and 'results' in js and 'headings' in js:
            table_like = [js['headings']] + list(js.get('results') or [])
            return list_table_to_json(table_like)
        return []
    if isinstance(payload, dict) and 'results' in payload and 'headings' in payload:
        table_like = [payload['headings']] + list(payload.get('results') or [])
        return list_table_to_json(table_like)
    return []

def tw_search_all(search, query: str, limit: int = 500) -> List[Dict[str, Any]]:
    # Use format='object' so headings are included; bulk=True by default to collect all
    resp = search.search({'query': query}, format='object', limit=limit)
    return to_rows(resp)

def get_json(resp_or_obj: Any) -> Any:
    if hasattr(resp_or_obj, 'json'):
        try:
            return resp_or_obj.json()
        except Exception:
            return {}
    return resp_or_obj

def get_credential_map(twcreds_handle) -> Dict[str, Dict[str, Any]]:
    resp = twcreds_handle.get_vault_credentials
    items = get_json(resp) or []
    mapping: Dict[str, Dict[str, Any]] = {}
    if isinstance(items, list):
        for c in items:
            if not isinstance(c, dict):
                continue
            uuid = str(c.get('uuid') or '').split('/')[-1]
            if not uuid:
                continue
            mapping[uuid] = {
                'label': c.get('label'),
                'username': c.get('username')
                            or c.get('snmp.v3.securityname')
                            or c.get('aws.access_key_id')
                            or c.get('azure.application_id'),
                'index': c.get('index'),
            }
    return mapping

def flatten_list(value):
    if value is None:
        return []
    if isinstance(value, list):
        out = []
        for v in value:
            if isinstance(v, list):
                out.extend(v)
            else:
                out.append(v)
        return out
    return [value]

def pick_latest(rows: List[Dict[str, Any]], key: str) -> Dict[str, Any]:
    def _key(r):
        v = r.get(key)
        try:
            return int(v) if v is not None else -1
        except Exception:
            return -1
    return max(rows, key=_key) if rows else {}


## Queries

These TWQL queries mirror the DisMAL devices flow and the optional devices_with_cred lookup.

In [ ]:
# Trimmed last discovery access view with key fields needed for devices summary
qry_last_disco = '''
search DiscoveryAccess where endtime
ORDER BY discovery_endtime DESC
show
endpoint as 'DiscoveryAccess.endpoint',
#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.hostname as 'DeviceInfo.hostname',
#Member:List:List:DiscoveryRun.label as 'DiscoveryRun.label',
friendlyTime(discovery_starttime) as 'DiscoveryAccess.scan_starttime',
friendlyTime(discovery_endtime) as 'DiscoveryAccess.scan_endtime',
discovery_endtime as 'DiscoveryAccess.scan_endtime_raw',
whenWasThat(discovery_endtime) as 'DiscoveryAccess.when_last_scan',
(#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_access_method in ['windows', 'rcmd']
    and #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_slave
        or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.probed_os and 'Probe'
            or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_access_method) as 'DiscoveryAccess.current_access',
(kind(#Associate:Inference:InferredElement:)
    or inferred_kind
        or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.kind) as 'DiscoveryAccess.node_kind',
(#DiscoveryAccess:Metadata:Detail:SessionResult.credential and success
    or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_credential
    or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_slave
    or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.__preserved_last_credential) as 'DeviceInfo.last_credential',
#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.os_version as 'DeviceInfo.os_version',
(nodecount(traverse DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo
  traverse flags(include_destroyed) Primary:Inference:InferredElement: where not destroyed(#)) > 0) as 'DiscoveryAccess.host_node_updated',
end_state as 'DiscoveryAccess.end_state',
result as 'DiscoveryAccess.result'
'''

# devices_with_cred flow
qry_sessions_for_cred = lambda uuid: f"""
search SessionResult where credential = '{uuid}'
show
(#Detail:Metadata:DiscoveryAccess:DiscoveryAccess.#Associate:Inference:InferredElement:.name
  or #Detail:Metadata:DiscoveryAccess:DiscoveryAccess.#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.hostname) as 'device_name',
(kind(#Detail:Metadata:DiscoveryAccess:DiscoveryAccess.#Associate:Inference:InferredElement:)
  or #Detail:Metadata:DiscoveryAccess:DiscoveryAccess.inferred_kind
  or #Detail:Metadata:DiscoveryAccess:DiscoveryAccess.#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.kind) as 'inferred_node',
#Detail:Metadata:DiscoveryAccess:DiscoveryAccess.endpoint as 'scanned_endpoint',
credential as 'credential',
success as 'success',
message as 'message',
friendlyTime(time_index) as 'date_time',
#Detail:Metadata:DiscoveryAccess:DiscoveryAccess.#id as 'node_id'
"""

qry_di_for_cred = lambda uuid: f"""
search DeviceInfo where last_credential = '{uuid}' or last_slave = '{uuid}' or __preserved_last_credential = '{uuid}'
ORDER BY hostname
show
(hostname or sysname) as 'device_name',
kind as 'inferred_node',
#DiscoveryResult:DiscoveryAccessResult:DiscoveryResult:DiscoveryAccess.endpoint as 'scanned_endpoint',
#DiscoveryResult:DiscoveryAccessResult:DiscoveryResult:DiscoveryAccess.#id as 'da_node_id',
#DiscoveryResult:DiscoveryAccessResult:DiscoveryResult:DiscoveryAccess.reason as 'message',
method_success as 'success',
method_failure as 'failure',
friendlyTime(request_time) as 'date_time'
"""


## Run Report

When `DEVICES_WITH_CRED_UUID` is set, runs the devices_with_cred flow; otherwise generates the consolidated devices report.

In [ ]:
# --- Setup ---
cred_map = get_credential_map(twcreds)

print("\n=== Setup ===")
print(f"- Target: {target}")
print(f"- Credentials loaded: {len(cred_map)}")

# Expect DEVICES_WITH_CRED_UUID to be defined (or None/"")
cred_uuid = DEVICES_WITH_CRED_UUID if DEVICES_WITH_CRED_UUID else None

In [ ]:
print("\n=== Credential Filter ===" if cred_uuid else "\n=== No Credential Filter ===")
if cred_uuid:
    cred_detail_resp = twcreds.get_vault_credential(cred_uuid)
    cred_detail = get_json(cred_detail_resp) or {}
    print(f"- Credential UUID: {cred_uuid}")
    print(f"- Label: {cred_detail.get('label', cred_uuid)}")
    print(f"- Index: {cred_detail.get('index')}")

In [ ]:
print("\n=== Build Identities ===")

qry_device_ids = '''
                    search DiscoveryAccess
                    show
                    #::InferredElement:.name as 'InferredElement.name',
                    #::InferredElement:.hostname as 'InferredElement.hostname',
                    #::InferredElement:.local_fqdn as 'InferredElement.local_fqdn',
                    #::InferredElement:.sysname as 'InferredElement.sysname',
                    endpoint as 'DiscoveryAccess.endpoint',
                    #DiscoveryAccess:Endpoint:Endpoint:Endpoint.endpoint as 'Endpoint.endpoint',
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DiscoveredIPAddressList.#List:List:Member:DiscoveredIPAddress.ip_addr as 'DiscoveredIPAddress.ip_addr',
                    #::InferredElement:.__all_ip_addrs as 'InferredElement.__all_ip_addrs',
                    #::InferredElement:.#DeviceWithInterface:DeviceInterface:InterfaceOfDevice:NetworkInterface.ip_addr as 'NetworkInterface.ip_addr',
                    #::InferredElement:.#DeviceWithInterface:DeviceInterface:InterfaceOfDevice:NetworkInterface.fqdns as 'NetworkInterface.fqdns'
                    '''

id_rows = tw_search_all(twsearch, qry_device_ids) or []
print(f"- Identity rows: {len(id_rows)}")

endpoint_map = {}  # ep -> {"ips": set(), "names": set()}
ip_fields = [
    'DiscoveryAccess.endpoint', 'Endpoint.endpoint',
    'DiscoveredIPAddress.ip_addr', 'InferredElement.__all_ip_addrs',
    'NetworkInterface.ip_addr'
]
name_fields = [
    'InferredElement.name', 'InferredElement.hostname',
    'InferredElement.local_fqdn', 'InferredElement.sysname',
    'NetworkInterface.fqdns'
]

for rec in id_rows:
    ep = rec.get('DiscoveryAccess.endpoint')
    if not ep:
        continue
    data = endpoint_map.setdefault(ep, {'ips': set(), 'names': set()})
    for f in ip_fields:
        data['ips'].update([v for v in flatten_list(rec.get(f)) if v])
    for f in name_fields:
        data['names'].update([v for v in flatten_list(rec.get(f)) if v])

identities = []
for ep, sets in endpoint_map.items():
    ips = sorted(set(sets['ips']))
    names = sorted(set(sets['names']))
    identities.append({'originating_endpoint': ep, 'list_of_ips': ips, 'list_of_names': names})

print(f"- Unique endpoints: {len(endpoint_map)}")
print(f"- Identities: {len(identities)}")

In [ ]:
print("\n=== Fetch Last Discovery Access (Filtered) ===" if cred_uuid else "\n=== Fetch Last Discovery Access ===")

ld_rows = tw_search_all(twsearch, qry_last_disco) or []

if cred_uuid:
    want = str(cred_uuid).split('/')[-1].lower()
    ld_rows = [
        r for r in ld_rows
        if r.get('DeviceInfo.last_credential')
        and str(r.get('DeviceInfo.last_credential')).split('/')[-1].lower() == want
    ]

print(f"- Last Discovery rows: {len(ld_rows)}")

by_ep = {}
for r in ld_rows:
    ep = r.get('DiscoveryAccess.endpoint')
    if not ep:
        continue
    by_ep.setdefault(ep, []).append(r)

In [ ]:
from itertools import islice

print("\n=== Assemble Results ===")

rows_out = []

for ident in identities:
    relevant = []
    for ip in ident.get('list_of_ips') or []:
        relevant.extend(by_ep.get(ip, []))

    if not relevant:
        #print(f"{ident.get('originating_endpoint')} not relevant")
        continue

    # summary
    #print(f"- endpoint={ident.get('originating_endpoint')} | ips={len(ident.get('list_of_ips') or [])} | matches={len(relevant)}")

    # tiny sample (first 3 rows, selected fields only)
    sample_fields = ["DiscoveryAccess.endpoint", "DeviceInfo.hostname", "DiscoveryRun.label"]
    for r in islice(relevant, 3):
        mini = {k: r.get(k) for k in sample_fields}
        #print("  ", json.dumps(mini, ensure_ascii=False))

    def to_str_set(val):
        """Flatten lists/tuples, drop None/empty, cast to str, return a set."""
        if isinstance(val, (list, tuple)):
            return {str(x) for x in val if x}
        return {str(val)} if val else set()

    # Build unique values safely
    all_device_names = sorted({
        s for r in relevant for s in to_str_set(r.get('DeviceInfo.hostname'))
    })
    
    all_endpoints = sorted({
        s for r in relevant for s in to_str_set(r.get('DiscoveryAccess.endpoint'))
    })
    
    all_discovery_runs = sorted({
        s for r in relevant for s in to_str_set(r.get('DiscoveryRun.label'))
    })

    # credentials used across relevant rows
    all_creds = []
    for r in relevant:
        cu = r.get('DeviceInfo.last_credential')
        if cu:
            cu_s = str(cu).split('/')[-1].lower()
            cm = cred_map.get(cu_s, {})
            all_creds.append(f"{cm.get('label') or cu_s} ({cu_s})")
    all_creds = sorted({v for v in all_creds if v})

    # latest and latest successful
    latest = pick_latest(relevant, 'DiscoveryAccess.scan_endtime_raw')
    succ_rows = [r for r in relevant if r.get('DiscoveryAccess.host_node_updated')]
    latest_succ = pick_latest(succ_rows, 'DiscoveryAccess.scan_endtime_raw') if succ_rows else {}

    last_scanned_ip = latest.get('DiscoveryAccess.endpoint')
    last_identity = latest.get('DeviceInfo.hostname') or (all_device_names[0] if all_device_names else None)
    last_kind = latest.get('DiscoveryAccess.node_kind')

    last_cred_uuid = latest.get('DeviceInfo.last_credential')
    last_cred_uuid = str(last_cred_uuid).split('/')[-1].lower() if last_cred_uuid else None
    cred_info = cred_map.get(last_cred_uuid or '', {})

    ls_identity = latest_succ.get('DeviceInfo.hostname') if latest_succ else None
    ls_ip = latest_succ.get('DiscoveryAccess.endpoint') if latest_succ else None
    ls_cred = latest_succ.get('DeviceInfo.last_credential') if latest_succ else None
    ls_cred = str(ls_cred).split('/')[-1].lower() if ls_cred else None
    ls_cred_info = cred_map.get(ls_cred or '', {})

    rows_out.append([
        last_scanned_ip,
        (ident.get('list_of_names') or [None])[0] or last_identity,
        last_kind,
        all_device_names or None,
        all_endpoints or None,
        all_creds or None,
        all_discovery_runs or None,
        last_cred_uuid,
        cred_info.get('label'),
        cred_info.get('username'),
        latest.get('DiscoveryAccess.scan_starttime'),
        latest.get('DiscoveryRun.label'),
        latest.get('DiscoveryAccess.end_state'),
        latest.get('DiscoveryAccess.result'),
        latest.get('DiscoveryAccess.current_access'),
        ls_identity,
        ls_ip,
        ls_cred,
        ls_cred_info.get('label'),
        ls_cred_info.get('username'),
        latest_succ.get('DiscoveryAccess.scan_starttime') if latest_succ else None,
        latest_succ.get('DiscoveryRun.label') if latest_succ else None,
        latest_succ.get('DiscoveryAccess.end_state') if latest_succ else None,
    ])

headers = [
    'last_scanned_ip',
    'last_identity',
    'last_kind',
    'all_device_names',
    'all_endpoints',
    'all_credentials_used',
    'all_discovery_runs',
    'last_credential',
    'last_credential_label',
    'last_credential_username',
    'last_start_time',
    'last_run',
    'last_endstate',
    'last_result',
    'last_access_method',
    'last_successful_identity',
    'last_successful_ip',
    'last_successful_credential',
    'last_successful_credential_label',
    'last_successful_credential_username',
    'last_successful_start_time',
    'last_successful_run',
    'last_successful_endstate',
]
df_out = pd.DataFrame(rows_out, columns=headers)
display(df_out.head())

In [ ]:
df_out.insert(0, 'Discovery Instance', target)

print(f"- Rows assembled: {len(rows_out)}")
print(f"- Output shape: {df_out.shape}")

REPORT_NAME = 'devices'
display(df_out.head(10))

## Save to CSV

Writes the report to the standard output folder in the project root.

In [ ]:
OUTPUT_CSV = str(output_dir / f'{REPORT_NAME}.csv')
df_out.to_csv(OUTPUT_CSV, index=False)
print(f'Saved to {OUTPUT_CSV}')
